**(RAG) Retrival Augmented Generation - using Llama-2-7b, BAAI Embeddings, FAISS & Langchain**

**Dependencies**

In [ ]:
!pip install -q transformers langchain accelerate einops bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


In [ ]:

!pip install sentence-transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 638.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e9c4d40972207e70a5ff22762e2b41062ee335bbc1ec611bf153ffcb714d7346
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


**For this step: You need Create Your Hugging Face account Give your HF Key**

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.2 MB/s eta 0:00:00


**Mount Drive**

In [ ]:
from google.colab import drive
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="You seem to be using the pipelines sequentially on GPU.")

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


**Loading Llama-2-7B**
` You need to Apply for Llama-2-Access on Hugging face from Meta if you donot have.`
**link:** https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    eos_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature': 0.2})

**Setup Path to save Embeddings Locally**

In [ ]:
DB_FAISS_PATH = '/content/drive/MyDrive/Faisss'

**Loading Csv File**

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from typing import List, Optional, Dict


In [ ]:
import csv
from io import TextIOWrapper
from typing import Any, Dict, List, Optional, Sequence

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader
from langchain.document_loaders.helpers import detect_file_encodings
from langchain.document_loaders.unstructured import (
    UnstructuredFileLoader,
    validate_unstructured_version,
)

class CSVLoader2(BaseLoader):
    def __init__(
        self,
        file_paths: List[str],
        source_column: Optional[str] = None,
        csv_args: Optional[Dict] = None,
        encoding: Optional[str] = None,
    ):
        self.file_paths = file_paths
        self.source_column = source_column
        self.encoding = encoding
        self.csv_args = csv_args or {}

    def load(self) -> List[Document]:
        docs = []
        for file_path in self.file_paths:
            with open(file_path, newline="", encoding=self.encoding) as csvfile:
                csv_reader = csv.DictReader(csvfile, **self.csv_args)  # type: ignore
                for i, row in enumerate(csv_reader):
                    content = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in row.items())
                    try:
                        source = (
                            row[self.source_column]
                            if self.source_column is not None
                            else file_path
                        )
                    except KeyError:
                        raise ValueError(
                            f"Source column '{self.source_column}' not found in CSV file."
                        )
                    metadata = {"source": source, "row": i}
                    doc = Document(page_content=content, metadata=metadata)
                    docs.append(doc)
        return docs

In [ ]:
loader = CSVLoader2(

    file_paths = ['/content/drive/MyDrive/Fetch_files/brand_category.csv','/content/drive/MyDrive/Fetch_files/categories.csv','/content/drive/MyDrive/Fetch_files/offer_retailer.csv'],
    csv_args={
        "delimiter": ",",
    },
)

data = loader.load()

**Loading Embedding Model from Hugging Face**

```
BAAI/Bge-small-en from Hugging Face
```



In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.2k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

**Creating Embeddings & Save Locally**

In [ ]:
db = FAISS.from_documents(data, embeddings)

In [ ]:
db.save_local(DB_FAISS_PATH)

**Creating a Prompt for Customized Response**

In [ ]:
from langchain.prompts.prompt import PromptTemplate

custom_template = """You will be provided with a dataset comprising offers, along with associated metadata about retailers and brands.
Additionally, a dataset of supported brands and their corresponding product categories will be available. The goal is to develop a
tool that enables users to search for offers based on categories, brands, or retailers, and receive a list of relevant offers along
with the similarity score used in the matching process.

Acceptance Criteria:

If a user searches for a category (e.g., diapers), the tool should return a list of offers relevant to that category.
If a user searches for a brand (e.g., Huggies), the tool should return a list of offers relevant to that brand.
If a user searches for a retailer (e.g., Target), the tool should return a list of offers relevant to that retailer.
The tool should provide the similarity score used to measure the text input's similarity with each offer.

Your Response should be very conscise and to the Point.

-----------------
Context:
{context}

-------------------

{chat_history}

Here are sample queries based on the provided dataset:

1. Search by Category (e.g., Diapers):

User Input: "Find offers related to diapers."
Expected Output:
Offer: "Spend $50 on a Full-Priced new Club Membership"
Retailer: SAMS CLUB
Brand: SAMS CLUB
Product Category: Red Pasta Sauce

Offer: "Egglife Egg White Wraps at Aldi"
Retailer: ALDI
Brand: EGGLIFE
Product Category: Frozen Desserts

2. Search by Brand (e.g., Huggies):

User Input: "Show offers from the brand Huggies."
Expected Output:
Offer: "Dr Pepper®, Regular OR Zero Sugar Strawberries and Cream 12 pack, at select retailers"
Retailer: UNITED SUPERMARKETS
Brand: DR PEPPER
Product Category: Cream

Offer: "Arnold, Brownberry, Oroweat® Small Slice bread, at Walmart"
Retailer: WALMART
Brand: ARNOLD BROWNBERRY OROWEAT
Product Category: Coffee

3. Search by Retailer (e.g., Target):

User Input: "Explore offers available at Walmart."
Expected Output:
Offer: "Arnold, Brownberry, Oroweat® Small Slice bread, at Walmart"
Retailer: WALMART
Brand: ARNOLD BROWNBERRY OROWEAT
Product Category: Coffee

4. Additional Query (Combination):

User Input: "Any offers on Frozen Fruits from Barilla?"
Expected Output:
Offer: "Barilla® Pesto Sauce"
Brand: BARILLA
Product Category: Frozen Fruits

---------------------

Question: {question}

Answer:"""
prompt = PromptTemplate(template=custom_template, input_variables=["context", "chat_history", "question"])


**Setting up Conversation Retrieval Chain**

In [ ]:
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), return_source_documents=True, combine_docs_chain_kwargs={
        'prompt': prompt,
    })



In [ ]:
torch.cuda.empty_cache()

**Running Queries and Getting Results:**

In [ ]:
def run_queries_and_display_results(chain, chat_history, queries):
    results = []
    for query in queries:
        result = chain({"question": query, "chat_history": chat_history})
        results.append(result['answer'])
        chat_history += [(query, result['answer'])]

    return results, chat_history

#user_input = "Diapers"
#user_input = "DOLLAR GENERAL STORE"
#user_input = "Eggs"
#user_input="LOREAL PARIS HAIR COLOR"

user_input = "Diapers"
queries = [f"Show offers related to {user_input}"]


results, chat_history = run_queries_and_display_results(chain, [], queries)
# Extract questions and answers
qa_pairs = [(query, result) for query, result in zip(queries, results)]

# Display questions and answers
for question, answer in qa_pairs:
    print(f"Question: {question}\nAnswer: {answer}\n")


Question: Show offers related to Diapers
Answer:  Offer: "Spend $50 on a Full-Priced new Club Membership"
Retailer: SAMS CLUB
Brand: SAMS CLUB
Product Category: Red Pasta Sauce
















































